# **Messages:**

Let's focus on working with messages in a LangGraph graph by using a special reducer called add_messages. The example shows how you can manage message states effectively by adding, overwriting, and removing messages in a LangGraph workflow.

In [2]:
%%capture --no-stderr
%pip install --quiet -U langchain_core langgraph

## **Setting Up Message State with Reducers:**

### **Defining a Custom TypedDict:**

```
from langgraph.graph.message import add_messages
from typing_extensions import TypedDict

class CustomMessagesState(TypedDict):
    messages: Annotated[list[AnyMessage], add_messages]
    added_key_1: str
    added_key_2: str
```

This defines a custom state schema where:

* messages is a list of messages, and the add_messages reducer will control how new messages are added.

* added_key_1 and added_key_2 are placeholders for additional state keys that can be added to your state dictionary, beyond just messages.

Here, we import MessagesState, which has a built-in key called messages, designed for working with a list of messages. Additionally, add_messages is imported. It's a reducer used to append messages to the messages list.

### **Using MessagesState:**

In [22]:
from langgraph.graph import MessagesState

class ExtendedMessagesState(MessagesState):
    added_key_1: str
    added_key_2: str

This shows that you can extend the built-in MessagesState for more flexibility. MessagesState already contains a messages key with the add_messages reducer, so it's a shortcut to handling message states.

In [23]:
from langgraph.graph.message import add_messages
from langchain_core.messages import AIMessage, HumanMessage

# Initial state
initial_messages = [AIMessage(content="Hello! How can I assist you?", name="Model"),
                    HumanMessage(content="I'm looking for information on Generative AI.", name="User")
                   ]

# New message to add
new_message = AIMessage(content="Sure, I can help with that. What specifically are you interested in?", name="Model")

# Capture the updated list by reassigning the return of add_messages
initial_messages = add_messages(initial_messages, new_message)

# Print each message to verify the updated state
for m in initial_messages:
    m.pretty_print()

================================== Ai Message ==================================
Name: Model

Hello! How can I assist you?
================================ Human Message =================================
Name: User

I'm looking for information on Generative AI.
================================== Ai Message ==================================
Name: Model

Sure, I can help with that. What specifically are you interested in?


### **Re-writing Messages:**

* Messages in LangGraph have unique IDs. If a new message has the same ID as an existing one (in this case, id="2"), it overwrites the old message.

* This feature is helpful when updating or correcting previously sent messages.

In [24]:
# Initial state with IDs
initial_messages = [AIMessage(content="Hello! How can I assist you?", name="Model", id="1"),
                    HumanMessage(content="I'm looking for information on Generative AI.", name="User", id="2")
                   ]

# New message with the same ID as an existing message
new_message = HumanMessage(content="I'm looking for information on Langgraph, specifically", name="User", id="2")

# Apply the additions and reassign the updated list to 'messages'
initial_messages = add_messages(initial_messages, new_message)

# Print each message to verify the updated state
for m in initial_messages:
    m.pretty_print()

================================== Ai Message ==================================
Name: Model

Hello! How can I assist you?
================================ Human Message =================================
Name: User

I'm looking for information on Langgraph, specifically


### **Removing Messages:**

* RemoveMessage is used to delete messages from the list.

* A loop isolates messages to delete by specifying their id. In this example, we remove the first two messages by creating RemoveMessage objects.

* The add_messages reducer is reused to apply the deletion and update the state.

In [25]:
from langchain_core.messages import AIMessage, HumanMessage, RemoveMessage
from langgraph.graph.message import add_messages

# Message list
messages = [
    AIMessage("Hi.", name="Bot", id="1"),
    HumanMessage("Hi.", name="User", id="2"),
    AIMessage("So you said you were researching LLMs?", name="Bot", id="3"),
    HumanMessage("Yes, I know about LLMs. But what others should I learn about?", name="user", id="4")
]

# Isolate messages to delete (first two messages)
delete_messages = [RemoveMessage(id=m.id) for m in messages[:-2]]

# Apply the deletions and reassign the updated list to 'messages'
messages = add_messages(messages, delete_messages)

# Display remaining messages to verify the deletion
for m in messages:
    m.pretty_print()

================================== Ai Message ==================================
Name: Bot

So you said you were researching LLMs?
================================ Human Message =================================
Name: user

Yes, I know about LLMs. But what others should I learn about?


**Identify Messages to Delete:** delete_messages is a list of RemoveMessage objects created by iterating over messages[:-2], which selects the first two messages in the list.